In [35]:
import pickle
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob
import contractions
import emoji
from scipy.sparse import hstack

In [36]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('punkt_tab')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\krish\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\krish\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\krish\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\krish\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [37]:
tfidf_path = "K:/Computer Science/AIMLDL/Brand_Reputation_Management_System/sentiment_analysis/feature_extraction/features/tfidf.pkl"
    
with open(tfidf_path, 'rb') as file:
    tfidf = pickle.load(file)

label_encoder_path = "K:/Computer Science/AIMLDL/Brand_Reputation_Management_System/sentiment_analysis/feature_extraction/features/label_encoder.pkl"
with open(label_encoder_path, 'rb') as f:
    label_encoder = pickle.load(f)

gender_encoder_path = "K:/Computer Science/AIMLDL/Brand_Reputation_Management_System/sentiment_analysis/feature_extraction/features/gender_encoder.pkl"
with open(gender_encoder_path, 'rb') as f:
    gender_encoder = pickle.load(f)

model_path = "K:/Computer Science/AIMLDL/Brand_Reputation_Management_System/sentiment_analysis/model_building/models/XGBClassifier.pkl"
with open(model_path, 'rb') as file:
    model = pickle.load(file)

In [38]:
def preprocess(text):
    text = contractions.fix(text)
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = emoji.replace_emoji(text, replace='')
    text = re.sub(r"[^a-zA-Z\s]", "", text)    
    text = text.lower()
    text = str(TextBlob(text).correct())
    words = word_tokenize(text)
    words = [word for word in words if word not in stop_words]
    words = [lemmatizer.lemmatize(word) for word in words]
    preprocessed_text = " ".join(words)
    
    return preprocessed_text

from scipy.sparse import csr_matrix

def predict_sentiment(text, age, gender):
    preprocessed_text = preprocess(text)
    custom_input_tfidf = tfidf.transform([preprocessed_text])

    # Normalize gender input and validate
    gender = gender.lower()
    if gender not in gender_encoder.classes_:
        raise ValueError(f"Unsupported gender label: '{gender}'. Please use one of {gender_encoder.classes_}.")
    gender_encoded = gender_encoder.transform([gender])[0]

    # Convert age and gender_encoded to sparse format
    metadata_features = csr_matrix([[age, gender_encoded]])

    # Combine tfidf features and metadata
    combined_features = hstack([custom_input_tfidf, metadata_features])

    # Predict sentiment
    prediction = model.predict(combined_features)
    predicted_sentiment = label_encoder.inverse_transform(prediction)[0]
    return predicted_sentiment


In [39]:
custom_text = "I've used this product for a while now, and it's been reliable. However, it's not the best I've ever used"
age = 20
gender = "female"
predicted_sentiment = predict_sentiment(custom_text, age, gender)
print(f"Predicted sentiment: {predicted_sentiment}")

Predicted sentiment: positive
